In [18]:
!pip install langchain faiss-cpu chromadb pypdf
!pip install PyPDF2
!pip install -U langchain-community
!pip install sentence-transformers
!pip install ipywidgets
!pip install langchain-openai
!pip install rouge-score


In [19]:
from langchain_community.document_loaders import NotebookLoader
loader = NotebookLoader(
    "/content/drive/MyDrive/3rd sem/data 255/hw/hw11.ipynb",
    include_outputs=True,
    max_output_length=20,
    remove_newline=True,
)
pages = loader.load_and_split()


In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# was 512 and 20
text_splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=200)
texts = text_splitter.split_documents(pages)


In [21]:
from google.colab import userdata

gpt_key = userdata.get('OPENAI_KEY')

In [22]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=gpt_key)

<ipython-input-22-aa90dac000ca>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=gpt_key)


In [23]:
#from langchain.embeddings import HuggingFaceEmbeddings

#model_name = "sentence-transformers/all-MiniLM-L6-v2"

#embeddings = HuggingFaceEmbeddings(model_name=model_name)
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=gpt_key)


In [24]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(texts, embeddings)

# Create a retriever from the FAISS index
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})



In [27]:
from langchain.chains import (
    create_history_aware_retriever,
    create_retrieval_chain,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI


# Contextualize question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Answer question
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(

    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
         ("human", "{input}"),
        ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain
)

In [28]:
from langchain_core.messages import HumanMessage

In [29]:
chat_history = []
query = "What libraries were pip installed in the document?"
output = rag_chain.invoke({"input":query, "chat_history":chat_history})
print(output)
chat_history.extend([HumanMessage(content=query), output["answer"]])

{'input': 'What libraries were pip installed in the document?', 'chat_history': [], 'context': [Document(metadata={'source': '/content/drive/MyDrive/3rd sem/data 255/hw/hw11.ipynb'}, page_content="'code' cell: '['pip install PyPDF2']'"), Document(metadata={'source': '/content/drive/MyDrive/3rd sem/data 255/hw/hw11.ipynb'}, page_content="'code' cell: '['pip install openai==0.28']'"), Document(metadata={'source': '/content/drive/MyDrive/3rd sem/data 255/hw/hw11.ipynb'}, page_content="with output: '['Collecting PyPDF2', '  Downloading pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)', 'Downloading pypdf2-3.0.1-py3-none-any.whl (232 kB)', '\\x1b[?25l   \\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\\x1b \\x1b0.0/232.6 kB\\x1b \\x1b?\\x1b eta \\x1b-:--:--\\x1b\\r\\x1b[2K   \\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\\x1b\\x1b╸\\x1b\\x1b━\\x1b \\x1b225.3/232.6 kB\\x1b \\x1b8.5 MB/s\\x1b eta \\x1b0:00:01\\x1b\\r\\x1b[2K   \\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\\x1b \\x1b232.6/232.6 kB\\x

In [30]:
print(output["answer"])

The libraries that were pip installed in the document are PyPDF2 and openai==0.28.


In [31]:

query = "How is the PDF file being read?"
output = rag_chain.invoke({"input":query, "chat_history":chat_history})
print(output["answer"])
chat_history.extend([HumanMessage(content=query), output["answer"]])

The PDF file is being read using the PyPDF2 library in the document.


In [32]:

query = "How is the website being read?"
output = rag_chain.invoke({"input":query, "chat_history":chat_history})
print(output["answer"])
chat_history.extend([HumanMessage(content=query), output["answer"]])

The website is being read by sending an HTTP request to the website and then parsing the content using BeautifulSoup in the document.


In [33]:

query = "What is the notebook doing?"
output = rag_chain.invoke({"input":query, "chat_history":chat_history})
print(output["answer"])
chat_history.extend([HumanMessage(content=query), output["answer"]])

The notebook is demonstrating how to rewrite and answer questions based on content from a given PDF and website. It shows the process of reading a PDF file using PyPDF2, extracting information, and then answering questions related to the content. Additionally, it explains how to read website text by sending an HTTP request, parsing the content, and generating responses based on the information retrieved.


In [34]:

query = "Summarize the chat history."
output = rag_chain.invoke({"input":query, "chat_history":chat_history})
print(output["answer"])

The conversation involves the assistant helping the student solve various math problems, including calculating the modulus of a complex number and solving a quadratic equation. The assistant also rewrites and solves a math problem from a hypothetical SAT practice test found on a website, which involves determining the amount of premium gasoline sold at a gas station. The student is guided through setting up equations and solving them to find the solutions to the given math problems.
